# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities_complete.csv")

# Display sample data
city_data_df.sample(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
404,404,morehead city,34.7229,-76.7260,20.01,86,0,5.14,US,1701649363
328,328,murzuq,25.9155,13.9184,16.29,19,2,4.57,LY,1701649311
496,496,verkhnevilyuysk,63.4453,120.3167,-48.51,100,78,0.28,RU,1701649418
471,471,wuda,39.4994,106.7117,-0.87,38,46,1.13,CN,1701649404
377,377,yuzhno-kurilsk,44.0306,145.8556,1.32,64,38,3.03,RU,1701649344
345,345,saint-philippe,-21.3585,55.7679,20.77,72,55,1.81,RE,1701649324
389,389,irthlingborough,52.3267,-0.6113,7.51,99,100,3.77,GB,1701649352
17,17,waitangi,-43.9535,-176.5597,14.34,89,100,2.68,NZ,1701649107
326,326,bamboo flat,11.7000,92.7167,27.05,89,75,3.60,IN,1701649310
301,301,newman,37.3138,-121.0208,16.68,46,100,1.48,US,1701649295


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
# %%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = 'City',
    size = 'Humidity',
    scale = 0.7,
    frame_width = 800,
    frame_height = 600    
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp']< 29) &
                           (city_data_df['Max Temp']> 20) &
                           (city_data_df['Wind Speed']< 4.5) &
                           (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna(how='any')

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kish,26.5578,54.0194,23.15,60,0,0.00,IR,1701649096
28,28,harper,4.3750,-7.7169,26.17,86,0,3.50,LR,1701649115
76,76,acapulco de juarez,16.8634,-99.8901,28.90,70,0,2.06,MX,1701648947
172,172,cabo san lucas,22.8909,-109.9124,26.95,52,0,2.06,MX,1701648696
174,174,abu dhabi,24.4667,54.3667,26.64,72,0,0.00,AE,1701648999
178,178,pedernales,18.0384,-71.7440,25.96,78,0,2.53,DO,1701649213
272,272,massawa,15.6097,39.4500,27.70,69,0,1.32,ER,1701649275
320,320,port lincoln,-34.7333,135.8667,23.87,47,0,3.73,AU,1701649307
324,324,atar,20.5169,-13.0499,23.01,32,0,2.55,MR,1701649309
380,380,ormara,25.2088,64.6357,23.63,51,0,3.85,PK,1701649346


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name", to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kish,IR,26.5578,54.0194,60,
28,harper,LR,4.3750,-7.7169,86,
76,acapulco de juarez,MX,16.8634,-99.8901,70,
172,cabo san lucas,MX,22.8909,-109.9124,52,
174,abu dhabi,AE,24.4667,54.3667,72,
178,pedernales,DO,18.0384,-71.7440,78,
272,massawa,ER,15.6097,39.4500,69,
320,port lincoln,AU,-34.7333,135.8667,47,
324,atar,MR,20.5169,-13.0499,32,
380,ormara,PK,25.2088,64.6357,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
params = {
    "categories":categories,
    "apiKey":geoapify_key,
    'limit':20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
kish - nearest hotel: هتل فارابی
harper - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
cabo san lucas - nearest hotel: Comfort Rooms
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
pedernales - nearest hotel: Hostal Doña Chava
massawa - nearest hotel: Seghen International Hotel فندق دولي سيغن
port lincoln - nearest hotel: Boston Hotel
atar - nearest hotel: فندق سكليل
ormara - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova


,City,Country,Lat,Lng,Humidity,Hotel Name
0,kish,IR,26.5578,54.0194,60,هتل فارابی
28,harper,LR,4.3750,-7.7169,86,No hotel found
76,acapulco de juarez,MX,16.8634,-99.8901,70,Hotel del Valle
172,cabo san lucas,MX,22.8909,-109.9124,52,Comfort Rooms
174,abu dhabi,AE,24.4667,54.3667,72,Marriott Downtown Abu-Dhabi
178,pedernales,DO,18.0384,-71.7440,78,Hostal Doña Chava
272,massawa,ER,15.6097,39.4500,69,Seghen International Hotel فندق دولي سيغن
320,port lincoln,AU,-34.7333,135.8667,47,Boston Hotel
324,atar,MR,20.5169,-13.0499,32,فندق سكليل
380,ormara,PK,25.2088,64.6357,51,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_map_df = hotel_df.loc[hotel_df['Hotel Name'] != 'No hotel found']
hotel_map_df

hotel_map = hotel_map_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = 'City',
    size = 'Humidity',
    hover_cols = ['Hotel Name','Country'],
    frame_width = 800,
    frame_height = 600    
)
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)